In [1]:
from tkinter import *
from tkinter import ttk
import numpy as np
import pandas as pd
from scipy.stats import norm
from yahoofinancials import YahooFinancials
import plotly.express as px
import scipy.optimize as opt
import pandas_datareader.data as web
import datetime as date
import mplfinance as mpf
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt
import data as dt
import Funciones as f
from scipy.stats import norm
import scipy.stats as sta

Basic Materials


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  3.33it/s]


In [2]:
tickerr = dt.Tickets[:7]

In [3]:
#tickerr = ["AAPL", "FB", "SHOP", "C", "AMZN", "MSFT", "EPD", "KO", "TSLA","WMT"]
prices = web.DataReader(name = tickerr, data_source = "yahoo",
              start= "2022-01-01")["Adj Close"].dropna()

In [4]:
#pip install --upgrade yfinance

In [5]:
resultado=np.zeros(len(tickerr))
for i in range(len(tickerr)):
    datos=yf.download(tickerr[i], period='1d', interval='1m')
    price=datos.iloc[-1,3]
    resultado[i]=price

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
prices.iloc[-1,:] = resultado


In [7]:
#Analisis RSI 
# data = yf.download(tickers, period='1d', interval='1m')
data = prices

RSII = pd.DataFrame()
for i in range(len(tickerr)):
    
    delta = data.iloc[:,i].diff(1)
    delta.dropna(inplace=True)

    positive = delta.copy()
    negative = delta.copy()

    positive[positive < 0 ]=0
    negative[negative > 0 ]=0
    days = 14
    avrg_gain = positive.rolling(window=days).mean()
    avrg_loss = abs(negative.rolling(window=days).mean())

    rltv_strength = avrg_gain/avrg_loss
    RSI = 100-(100/(1+rltv_strength))

    combined = pd.DataFrame()
    combined["Close"] =  data.iloc[:,i]
    combined["RSI"] = RSI
    RSII[i] = RSI

    
RSII  = pd.DataFrame(RSII)
RSII.columns = tickerr
RSI_HOY = RSII.iloc[len(RSII)-1,:]

In [8]:
RSI_HOY=pd.DataFrame(RSI_HOY)

In [9]:
ws  = Tk()
ws.title('RSI Analysis')
ws.geometry('500x500')
#ws['bg'] = '#AC99F2'
game_frame = Frame(ws)
game_frame.pack()
my_game = ttk.Treeview(game_frame)
my_game['columns'] = ('TICKER','STATUS','PRICE')

my_game.column("#0", width=0,  stretch=NO)
my_game.column("TICKER",anchor=CENTER, width=80)
my_game.column("STATUS",anchor=CENTER,width=80)
my_game.column("PRICE",anchor=CENTER,width=80)

my_game.heading("#0",text="",anchor=CENTER)
my_game.heading("TICKER",text="TICKER",anchor=CENTER)
my_game.heading("STATUS",text="STATUS",anchor=CENTER)
my_game.heading("PRICE",text="PRICE",anchor=CENTER)

for i in range(len(tickerr)-1):
    if(RSI_HOY.iloc[i,0]<30):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'BUY',np.round((resultado[i]),4)))
    if(RSI_HOY.iloc[i,0]>70):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'SELL',np.round((resultado[i]),4)))
    if((RSI_HOY.iloc[i,0]>=30) & (RSI_HOY.iloc[i,0]<=70)):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'HOLD',np.round((resultado[i]),4)))

my_game.pack()

import tkinter

Etiqueta = ttk.Label(ws,text='Select your stock')
Etiqueta.pack()

n = tkinter.StringVar(ws)
n.set(tickerr[0])

seleccion = tkinter.OptionMenu(ws,n,*tickerr)
seleccion.pack()

def Close():
    a = n.get()
    ws.destroy()

exit_button = tkinter.Button(ws, text="Choose", command= lambda :Close())
exit_button.pack(pady=20)

    
ws.mainloop()


activo = n.get()

In [10]:
resultado=np.zeros(len(activo))
datos=yf.download(activo, period='1d', interval='1m')
price=datos.iloc[-1,3]
st = price   

[*********************100%***********************]  1 of 1 completed


In [11]:
def griegas(r,sigma,st,k,T=1,t=0):
    d_c = f.delta_call(St,k,T,t,r,sigma)
    d_p = f.delta_put(St,k,T,t,r,sigma)
    v   = f.vega(St,k,T,t,r,sigma)
    t_c = f.theta_call(St,k,T,t,r,sigma)
    t_p = f.theta_put(St,k,T,t,r,sigma)
    r_c = f.rho_call(St,k,T,t,r,sigma)
    r_p = f.rho_put(St,k,T,t,r,sigma)
    g   = f.gamma(St,k,T,t,r,sigma)
    van = f.vanna(St,k,T,t,r,sigma)
    vol = f.volga(St,k,T,t,r,sigma)
    col = f.color(St,k,T,t,r,sigma)
    
    
    return d_c,d_p,v,t_c,t_p,r_c,r_p,g,van,vol,col

In [12]:
#ITERACIÓN
import time 


contador = 0
while contador < 5:
    

    resultado=np.zeros(len(activo))
    datos=yf.download(activo, period='1d', interval='1m')
    price=datos.iloc[-1,3]
    st = price 

    r = .0852
    T = 1
    t = 0
    ret_log = np.log(datos.iloc[:,3]/datos.iloc[:,3].shift(1)).dropna()
    cambio_porcentual_anual = ret_log.iloc[-1]*(252)
    sigma = np.std(ret_log)*np.sqrt(252)
    St = st 
    k = [St,St*(1+sigma),St*(1+2*sigma),St*(1-sigma),St*(1-2*sigma)]

    d1= [(np.log(St/k[i])+(r +0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t)) for i in range(0,len(k))]
    d2= [(np.log(St/k[i])+(r -0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t)) for i in range(0,len(k))]

    precio_justo_call = [st*sta.norm.cdf(d1[i],0,1)-k[i]*np.exp(-r*(T-t))*sta.norm.cdf(d2[i],0,1) for i in range(0,len(d1))]
    precio_justo_put = [k[i]*np.exp(-r*(T-t))*sta.norm.cdf(-d2[i],0,1)-st*sta.norm.cdf(-d1[i],0,1) for i in range(0,len(d1))]


    k = np.round(k,3)
    precio_justo_call = precio_justo_call
    precio_justo_put = precio_justo_put


    a = [griegas(r,sigma,st,k[i],T=1,t=0) for i in range(0,len(k))]
    a = a


    k1 = [0,k[0],precio_justo_call[0],precio_justo_put[0],a[0][0],a[0][1],a[0][2],a[0][3],a[0][4],
         a[0][5],a[0][6],a[0][7],a[0][8],a[0][9],a[0][10]]

    k2 = [0,k[1],precio_justo_call[1],precio_justo_put[1],a[1][0],a[1][1],a[1][2],a[1][3],a[1][4],
         a[1][5],a[1][6],a[1][7],a[1][8],a[1][9],a[1][10]]

    k3 = [0,k[2],precio_justo_call[2],precio_justo_put[2],a[2][0],a[2][1],a[2][2],a[2][3],a[2][4],
         a[2][5],a[2][6],a[2][7],a[2][8],a[2][9],a[2][10]]

    k4 = [0,k[3],precio_justo_call[3],precio_justo_put[3],a[3][0],a[3][1],a[3][2],a[3][3],a[3][4],
         a[3][5],a[3][6],a[3][7],a[3][8],a[3][9],a[3][10]]


    k5 = [0,k[4],precio_justo_call[4],precio_justo_put[4],a[4][0],a[4][1],a[4][2],a[4][3],a[4][4],
         a[4][5],a[4][6],a[4][7],a[4][8],a[4][9],a[4][10]]


    ws  = Tk()
    ws.title('Griegas '+activo)

    #ws['bg'] = '#AC99F2'
    game_frame = Frame(ws)
    game_frame.pack()
    my_game = ttk.Treeview(game_frame)
    my_game['columns'] = ('p','k','call','put','Delta_Call','Delta_put','Vega','Theta_call'
                               ,'Theta_put','rho_call','rho_put','gamma',
                              'vanna','volga','color')

    my_game.column("p", width=0,  stretch=NO)
    my_game.column("k",anchor=CENTER, width=80)
    my_game.column("call",anchor=CENTER, width=80)
    my_game.column("put",anchor=CENTER, width=80)
    my_game.column("Delta_Call",anchor=CENTER, width=80)
    my_game.column("Delta_put",anchor=CENTER,width=80)
    my_game.column("Vega",anchor=CENTER,width=80)
    my_game.column("Theta_call",anchor=CENTER,width=80)
    my_game.column("Theta_put",anchor=CENTER,width=80)
    my_game.column("rho_call",anchor=CENTER,width=80)
    my_game.column("rho_put",anchor=CENTER,width=80)
    my_game.column("gamma",anchor=CENTER,width=80)
    my_game.column("vanna",anchor=CENTER,width=80)
    my_game.column("volga",anchor=CENTER,width=80)
    my_game.column("color",anchor=CENTER,width=80)

    my_game.heading("p",text=" ",anchor=CENTER)
    my_game.heading("k",text="k",anchor=CENTER)
    my_game.heading("call",text="call",anchor=CENTER)
    my_game.heading("put",text="put",anchor=CENTER)
    my_game.heading("Delta_Call",text="Delta_Call",anchor=CENTER)
    my_game.heading("Delta_put",text="Delta_put",anchor=CENTER)
    my_game.heading("Vega",text="Vega",anchor=CENTER)
    my_game.heading("Theta_call",text="Theta_call",anchor=CENTER)
    my_game.heading("Theta_put",text="Theta_put",anchor=CENTER)
    my_game.heading("rho_call",text="rho_call",anchor=CENTER)
    my_game.heading("rho_put",text="rho_put",anchor=CENTER)
    my_game.heading("gamma",text="gamma",anchor=CENTER)
    my_game.heading("vanna",text="vanna",anchor=CENTER)
    my_game.heading("volga",text="volga",anchor=CENTER)
    my_game.heading("color",text="color",anchor=CENTER)

    my_game.insert(parent='',index='end',text='', values=(k1))
    my_game.insert(parent='',index='end',text='', values=(k2))
    my_game.insert(parent='',index='end',text='', values=(k3))
    my_game.insert(parent='',index='end',text='', values=(k4))
    my_game.insert(parent='',index='end',text='', values=(k5))
    my_game.pack()
    
    # Se actualizara la tabla 
    def cerrar():
        ws.destroy()
    
    ws.after(90000,cerrar)
    contador = contador+1 
    ws.mainloop()

    
        
    #ws.destroy()
    
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Coberturas 
opcciones call o put 